In [1]:
import pandas as pd
import numpy as np

In [2]:
# scaling factor, typically 1 million (hence the M in the normalization method names)
# here set to 10 (instead of 1000000) for more handy examples
SCALE = 10

In [3]:
# example data, consider the columns as three biological replicates, i.e. they should have identical values, but
# in reality, these numbers are heavily skewed

counts = pd.DataFrame(data=[['A',2,10,12,30],
                            ['B',4,20,25,60],
                            ['C',1,5,8,15],
                            ['D',10,0,0,1]],
                      columns=['gene','length','Rep1', 'Rep2', 'Rep3']).set_index(['gene','length'])
counts

,,Rep1,Rep2,Rep3
gene,length,,,
A,2,10,12,30
B,4,20,25,60
C,1,5,8,15
D,10,0,0,1


# RPKM
**R**eads **P**er **K**ilobase **M**illion
### 1) adjust for sequencing depth

In [4]:
# compute coverage per replicate
counts.sum()

Rep1     35
Rep2     45
Rep3    106
dtype: int64

In [5]:
# devide each gene/replicate read count by the replicate coverage
counts / counts.sum()

,,Rep1,Rep2,Rep3
gene,length,,,
A,2,0.285714,0.266667,0.283019
B,4,0.571429,0.555556,0.566038
C,1,0.142857,0.177778,0.141509
D,10,0.000000,0.000000,0.009434


In [6]:
# scale coverage
counts / (counts.sum() / SCALE)

,,Rep1,Rep2,Rep3
gene,length,,,
A,2,2.857143,2.666667,2.830189
B,4,5.714286,5.555556,5.660377
C,1,1.428571,1.777778,1.415094
D,10,0.000000,0.000000,0.094340


### 2) adjust for gene length
by dividing with gene length in kilo bases

note that we "transpose" the count table to operate on genes instead of replicates and "transpose" afterwards to restore original orientation

In [7]:
((counts / (counts.sum() / SCALE)).T / counts.index.get_level_values(1)).T

,,Rep1,Rep2,Rep3
gene,length,,,
A,2,1.428571,1.333333,1.415094
B,4,1.428571,1.388889,1.415094
C,1,1.428571,1.777778,1.415094
D,10,0.000000,0.000000,0.009434


# TPM
**T**ranscripts **P**er kilobase **M**illion
### 1) adjust for gene length
by dividing with gene length in kilo bases

In [8]:
(counts.T / counts.index.get_level_values(1)).T

,,Rep1,Rep2,Rep3
gene,length,,,
A,2,5.0,6.00,15.0
B,4,5.0,6.25,15.0
C,1,5.0,8.00,15.0
D,10,0.0,0.00,0.1


### 2) adjust for sequencing depth

In [9]:
x = (counts.T / counts.index.get_level_values(1)).T
x / x.sum()

,,Rep1,Rep2,Rep3
gene,length,,,
A,2,0.333333,0.296296,0.332594
B,4,0.333333,0.308642,0.332594
C,1,0.333333,0.395062,0.332594
D,10,0.000000,0.000000,0.002217


# DESeq2

In [10]:
# example data, consider the columns as three different conditions, i.e. they should have different values

counts = pd.DataFrame(data=[['Gene1',0,10,4],
                            ['Gene2',2,6,12],
                            ['Gene3',33,55,200]],
                      columns=['gene','Sample #1', 'Sample #2', 'Sample #3']).set_index(['gene'])
counts

,Sample #1,Sample #2,Sample #3
gene,,,
Gene1,0,10,4
Gene2,2,6,12
Gene3,33,55,200


### 1) log transform

In [11]:
np.log(counts)

/homes/sjanssen/miniconda3/envs/notebookServer/lib/python3.10/site-packages/pandas/core/internals/blocks.py:351: RuntimeWarning: divide by zero encountered in log
  result = func(self.values, **kwargs)


,Sample #1,Sample #2,Sample #3
gene,,,
Gene1,-inf,2.302585,1.386294
Gene2,0.693147,1.791759,2.484907
Gene3,3.496508,4.007333,5.298317


### 2) per gene, average across samples

In [12]:
np.log(counts).mean(axis=1)

/homes/sjanssen/miniconda3/envs/notebookServer/lib/python3.10/site-packages/pandas/core/internals/blocks.py:351: RuntimeWarning: divide by zero encountered in log
  result = func(self.values, **kwargs)


gene
Gene1        -inf
Gene2    1.656604
Gene3    4.267386
dtype: float64

### 3) filter genes with -inf at average

In [13]:
avg = np.log(counts).mean(axis=1).replace(-1*np.infty, None).dropna()
avg

gene
Gene2    1.656604
Gene3    4.267386
dtype: object

### 4) per cell, subtract average from log(counts)

In [14]:
(np.log(counts).T - avg).T

,Sample #1,Sample #2,Sample #3
gene,,,
Gene1,NaN,NaN,NaN
Gene2,-0.963457,0.135155,0.828302
Gene3,-0.770878,-0.260053,1.030931


### 5) calculate mean on the ratios for each sample

In [15]:
((np.log(counts).T - avg).T).mean()

/homes/sjanssen/miniconda3/envs/notebookServer/lib/python3.10/site-packages/pandas/core/internals/blocks.py:351: RuntimeWarning: divide by zero encountered in log
  result = func(self.values, **kwargs)


Sample #1   -0.867168
Sample #2   -0.062449
Sample #3    0.929617
dtype: float64

### 6) convert mean to normal space: exp(x)

In [16]:
np.exp(((np.log(counts).T - avg).T).mean())

/homes/sjanssen/miniconda3/envs/notebookServer/lib/python3.10/site-packages/pandas/core/internals/blocks.py:351: RuntimeWarning: divide by zero encountered in log
  result = func(self.values, **kwargs)


Sample #1    0.420140
Sample #2    0.939461
Sample #3    2.533538
dtype: float64

### 7) divide read counts by "ratio mean" scaling factor

In [17]:
counts / np.exp(((np.log(counts).T - avg).T).mean())

/homes/sjanssen/miniconda3/envs/notebookServer/lib/python3.10/site-packages/pandas/core/internals/blocks.py:351: RuntimeWarning: divide by zero encountered in log
  result = func(self.values, **kwargs)


,Sample #1,Sample #2,Sample #3
gene,,,
Gene1,0.000000,10.644401,1.578820
Gene2,4.760321,6.386640,4.736459
Gene3,78.545292,58.544204,78.940989


### 7b) round to neareast integer

In [18]:
round(counts / np.exp(((np.log(counts).T - avg).T).mean()))

/homes/sjanssen/miniconda3/envs/notebookServer/lib/python3.10/site-packages/pandas/core/internals/blocks.py:351: RuntimeWarning: divide by zero encountered in log
  result = func(self.values, **kwargs)


,Sample #1,Sample #2,Sample #3
gene,,,
Gene1,0.0,11.0,2.0
Gene2,5.0,6.0,5.0
Gene3,79.0,59.0,79.0
